In [8]:
# !pip3 install "camelot-py[base]"
import camelot
import pandas as pd
import numpy as np

url = "https://static-content.springer.com/esm/art%3A10.1038%2Fs41598-018-24725-0/MediaObjects/41598_2018_24725_MOESM3_ESM.pdf"
# 1) Download PDF
import requests, pathlib
pdf_path = pathlib.Path("lukassen_extracted/lukassen_supp_table_s2.pdf")
pdf_path.write_bytes(requests.get(url, timeout=60).content)

# 2) Extract tables from all pages (stream mode works well for this layout)
tables = camelot.read_pdf(str(pdf_path), pages="all", flavor="stream")

# 3) Concatenate and clean
dfs = []
for t in tables:
    df = t.df
    # Heuristics: drop empty rows/cols and try to fix split header lines
    df = df.dropna(how="all", axis=0).dropna(how="all", axis=1)
    dfs.append(df)

raw = pd.concat(dfs, ignore_index=True)

# 4) Normalize columns (rename to our schema; tweak if your PDF parsing shifts indices)
# Try to find the header row by matching "Gene name"
def row_has_gene_name(r):
    return r.astype(str).str.contains("Gene name").any()
hdr_idx = raw.index[raw.apply(row_has_gene_name, axis=1)][0]
raw.columns = raw.iloc[hdr_idx]
data = raw.iloc[hdr_idx+1:].copy()

# Filter out rows where "Gene name" col is "Supplementary Table S2" or empty
gene_col = "Gene name"
if gene_col in data.columns:
    # strip whitespace before filtering
    data[gene_col] = data[gene_col].astype(str).str.strip()
    data = data[data[gene_col] != "Supplementary Table S2"]
    # Remove rows where "Gene name" is empty, i.e. blank or only whitespace
    data = data[data[gene_col] != '']
    data = data[~data[gene_col].isna()]
    # Remove rows where "Gene name" is only numbers
    data = data[~data[gene_col].str.match(r"^\d+$")]
    # Remove rows where "Gene name" is a single character (of any type)
    data = data[data[gene_col].str.len() != 1]

# 5) Minimal post-processing (strip spaces, enforce 0/1 where applicable)
data = data.rename(columns={
    "Gene name":"Gene name","Early Sgonia":"Early Sgonia","Late Sgonia":"Late Sgonia",
    "Early Scytes":"Early Scytes","Late Scytes":"Late Scytes","Round Stids":"Round Stids",
    "Later Stids":"Later Stids","Sertoli":"Sertoli","Leydig":"Leydig","Method":"Method","Source":"Source"
})
for col in ["Early Sgonia","Late Sgonia","Early Scytes","Late Scytes","Round Stids","Later Stids","Sertoli","Leydig"]:
    if col in data.columns:
        data[col] = data[col].str.extract(r'(\d)').fillna("")
        data[col] = data[col].replace("", np.nan)  # Set empty strings to NaN

# Make sure ALL empty fields (empty string, only whitespace, or None) in the whole DataFrame are set to NaN:
data = data.applymap(lambda x: np.nan if pd.isna(x) or str(x).strip() == "" else x)

# 6) Save
data.to_csv("lukassen_extracted/lukassen_supp_table_s2_full.csv", index=False, na_rep='NaN')
print("Saved: lukassen_supp_table_s2_full.csv  | rows:", len(data))


Saved: lukassen_supp_table_s2_full.csv  | rows: 302


/var/folders/06/zfzddxwn4hd0ywccb30sl4qw0000gn/T/ipykernel_23454/1679382190.py:59: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: np.nan if pd.isna(x) or str(x).strip() == "" else x)
